* Faceリソースを作成する

# Detect Faces

* https://westus.dev.cognitive.microsoft.com/docs/services/563879b61984550e40cbbe8d/operations/563879b61984550f30395236
* FaceのDetectで FaceIDを取得
* Person GroupのCreateでグループを作成
* Person Group Person
    * Createで人を作成
    * Add Faceで顔を登録
* Person GroupでTrainを実行
* Faceで Identifyで誰かを識別

In [1]:
subscription_key = 'xxxx'

In [2]:
import urllib.parse, http.client, json

headers = {
    'Ocp-Apim-subscription-Key':subscription_key,
    'Content-Type':'application/json'
}

body = {
    'url':'https://upload.wikimedia.org/wikipedia/commons/thumb/0/0f/Angela_Merkel_2019_cropped.jpg/1200px-Angela_Merkel_2019_cropped.jpg'
}

params = urllib.parse.urlencode({
    'returnFaceId':True,
    'returnFaceLandmarks':True,
    'returnFaceAttributes':'age,gender,makeup,smile,glasses,accessories,emotion'
})

url = 'southcentralus.api.cognitive.microsoft.com'

endpoint = '/face/v1.0/detect?%s'

try:
    conn = http.client.HTTPSConnection(url)
    conn.request('POST', endpoint % params, str(body), headers)
    response = conn.getresponse()
    jsonData = response.read()
    data = json.loads(jsonData)
    detect_face_id = data[0]['faceId']
    print(json.dumps(data, sort_keys=True, indent=2))
    conn.close()
except Exception as ex:
    print(ex)

[
  {
    "faceAttributes": {
      "accessories": [],
      "age": 57.0,
      "emotion": {
        "anger": 0.0,
        "contempt": 0.0,
        "disgust": 0.0,
        "fear": 0.0,
        "happiness": 0.139,
        "neutral": 0.861,
        "sadness": 0.0,
        "surprise": 0.0
      },
      "gender": "female",
      "glasses": "NoGlasses",
      "makeup": {
        "eyeMakeup": false,
        "lipMakeup": false
      },
      "smile": 0.139
    },
    "faceId": "e9993669-530c-42b2-b83b-5adf0565f82c",
    "faceLandmarks": {
      "eyeLeftBottom": {
        "x": 479.1,
        "y": 549.3
      },
      "eyeLeftInner": {
        "x": 515.1,
        "y": 539.6
      },
      "eyeLeftOuter": {
        "x": 444.0,
        "y": 540.3
      },
      "eyeLeftTop": {
        "x": 476.1,
        "y": 518.8
      },
      "eyeRightBottom": {
        "x": 737.4,
        "y": 566.3
      },
      "eyeRightInner": {
        "x": 702.0,
        "y": 552.1
      },
      "eyeRightOuter": {
  

# Create a Group

In [3]:
group_id = 'random_people'

group_body = {
    'name':'Random People'
}

try:
    conn = http.client.HTTPSConnection(url)
    # putリクエスト
    # ドキュメントの通り、?はいらない
    conn.request('PUT', '/face/v1.0/persongroups/%s' % group_id, str(group_body), headers)
    response = conn.getresponse()
    data = response.read()
    print(data)
    conn.close()
except Exception as ex:
    print(ex)

b'{"error":{"code":"PersonGroupExists","message":"Person group \'random_people\' already exists."}}'


# Adding person to a Group

In [4]:
# 3回実行し3人を登録する
person_body = {
#     'name':'Elizabeth'
#     'name':'Bill'
      'name':'Angela'
}

person_endpoint = '/face/v1.0/persongroups/{0}/persons'.format(group_id)

try:
    conn = http.client.HTTPSConnection(url)
    conn.request('POST', person_endpoint, str(person_body), headers)
    response = conn.getresponse()
    jsonData = response.read()
    data = json.loads(jsonData)
    print(data)
    conn.close()
except Exception as ex:
    print(ex)

{'personId': 'e3ed90b3-e446-42af-88d0-5ddd6f1bce42'}


In [5]:
# personIdを確認
group_id = 'random_people'

endpoint = '/face/v1.0/persongroups/{0}/persons'.format(group_id)

try:
    conn = http.client.HTTPSConnection(url)
    # Getに変更、bodyを空白に変更
    conn.request('GET', endpoint, '', headers)
    response = conn.getresponse()
    jsonData = response.read()
    person_data = json.loads(jsonData)
    print(json.dumps(person_data, sort_keys=True, indent=2))
    conn.close()
except Exception as ex:
    print(ex)

[
  {
    "name": "Angela",
    "persistedFaceIds": [
      "25aa8d24-e2ec-401b-94a8-2d5c66ea88e7",
      "93adb9ed-b944-4911-9745-ef266ae394fc",
      "bedd81de-bb28-4da4-8b20-e0ca1710035c"
    ],
    "personId": "79f0b49d-3ba3-40ac-863a-115f7056b907"
  },
  {
    "name": "Bill",
    "persistedFaceIds": [
      "10fc469f-198a-4bef-8365-03399bd3b375",
      "1bb018e1-1787-4d51-8508-563360860a7b",
      "289841db-fd6d-428a-b94f-83d448ccd7db",
      "3a265ac8-9b3b-4e36-92de-718a2ffc7d7f",
      "49e4d91d-b547-4cc7-a2bd-560b44f4388d",
      "64c6e277-d1fc-44ec-96df-6defa421caf0",
      "7a379dfb-285d-4633-94c6-56bb642ab50c",
      "9bb3d38e-26dc-4b0f-925f-88e6c9c79603",
      "ad38193d-bdb7-489c-8b98-939c7e73493f"
    ],
    "personId": "93294a0c-0df9-4eb4-89af-fe9cc7e58c80"
  },
  {
    "name": "Elizabeth",
    "persistedFaceIds": [
      "0c8d9c77-cf68-4b26-ae85-ba2a5bf103fc",
      "87b3cfb3-d8eb-4f88-928c-8b49965e32d3",
      "923ea483-619d-4ddf-8757-79d79af6b770",
      "a1961af3-a24

In [6]:
# 削除したいときは以下のpersonIDを指定する
# try:
#     conn = http.client.HTTPSConnection(url)
#     conn.request('DELETE', endpoint + '/772e856e-6435-4642-bae3-9db110ace595', str(person_body), headers)
#     response = conn.getresponse()
#     jsonData = response.read()
#     print(jsonData)
#     conn.close()
# except Exception as ex:
#     print(ex)

# Adding faces to a person

In [7]:
import os

person_group_id = 'random_people'

url = 'southcentralus.api.cognitive.microsoft.com'

headers = {
    'Ocp-Apim-subscription-Key':subscription_key,
    'Content-Type':'application/octet-stream' # localのファイルをアップロードする為
}

for person in person_data:
    face_endpoint = '/face/v1.0/persongroups/{0}/persons/{1}/persistedFaces'.format(group_id, person['personId'])
    for file in os.listdir('people/'+person['name']):
        with open('people/{0}/{1}'.format(person['name'], file), 'rb') as body:
            conn = http.client.HTTPSConnection(url)
            conn.request('POST', face_endpoint, body, headers)
            response = conn.getresponse()
            jsonData = response.read()
            data = json.loads(jsonData)
            print(data)
            conn.close()

{'persistedFaceId': '042a1034-506d-459d-b5eb-09b0a6afa075'}
{'persistedFaceId': 'c293a7b5-1578-4f0c-ba63-94831377f734'}
{'persistedFaceId': 'cc930339-7d3c-4892-a86c-98fb0940e9d6'}
{'persistedFaceId': '5498b533-9d52-4acd-9fde-3fb357d0ca91'}
{'persistedFaceId': 'b4b8f4ea-1b87-47bf-81d2-de82c89780ad'}
{'persistedFaceId': '5e585606-cd7b-4a5d-9b43-c6e397859068'}
{'persistedFaceId': '6d699408-8698-4483-b19e-44b35f1c5fa8'}
{'persistedFaceId': 'ad27d423-28e4-457e-8d14-0358253abf92'}
{'persistedFaceId': 'fe61505c-3609-4465-bbe5-6e56bc46e457'}
{'persistedFaceId': '93faea86-d2a8-4cf9-af41-f6a212394d68'}
{'persistedFaceId': 'a7758a3c-c4d0-456d-821d-8557a7e734e2'}
{'persistedFaceId': '4e127d0a-be18-4166-adfa-ee10be64d614'}
{'persistedFaceId': 'a0236897-7e48-4448-9689-25b4fccba3a1'}
{'persistedFaceId': '840c3d50-44d5-404f-a12b-7ece6b74acf3'}
{'persistedFaceId': 'cc2e6d42-4715-4832-9dfd-ffbcf3565651'}


In [8]:
# FaceIdを確認
group_id = 'random_people'

endpoint = '/face/v1.0/persongroups/{0}/persons'.format(group_id)

try:
    conn = http.client.HTTPSConnection(url)
    # Getに変更、bodyを空白に変更
    conn.request('GET', endpoint, '', headers)
    response = conn.getresponse()
    jsonData = response.read()
    person_data = json.loads(jsonData)
    print(json.dumps(person_data, sort_keys=True, indent=2))
    conn.close()
except Exception as ex:
    print(ex)

[
  {
    "name": "Angela",
    "persistedFaceIds": [
      "042a1034-506d-459d-b5eb-09b0a6afa075",
      "25aa8d24-e2ec-401b-94a8-2d5c66ea88e7",
      "93adb9ed-b944-4911-9745-ef266ae394fc",
      "bedd81de-bb28-4da4-8b20-e0ca1710035c",
      "c293a7b5-1578-4f0c-ba63-94831377f734",
      "cc930339-7d3c-4892-a86c-98fb0940e9d6"
    ],
    "personId": "79f0b49d-3ba3-40ac-863a-115f7056b907"
  },
  {
    "name": "Bill",
    "persistedFaceIds": [
      "10fc469f-198a-4bef-8365-03399bd3b375",
      "1bb018e1-1787-4d51-8508-563360860a7b",
      "289841db-fd6d-428a-b94f-83d448ccd7db",
      "3a265ac8-9b3b-4e36-92de-718a2ffc7d7f",
      "49e4d91d-b547-4cc7-a2bd-560b44f4388d",
      "5498b533-9d52-4acd-9fde-3fb357d0ca91",
      "5e585606-cd7b-4a5d-9b43-c6e397859068",
      "64c6e277-d1fc-44ec-96df-6defa421caf0",
      "7a379dfb-285d-4633-94c6-56bb642ab50c",
      "9bb3d38e-26dc-4b0f-925f-88e6c9c79603",
      "ad38193d-bdb7-489c-8b98-939c7e73493f",
      "b4b8f4ea-1b87-47bf-81d2-de82c89780ad"
   

# Identifying people

In [13]:
# 学習を実施
import urllib.parse, http.client, json

url = 'southcentralus.api.cognitive.microsoft.com'

train_headers = {
    'Ocp-Apim-subscription-Key':subscription_key,
}

person_group_id = 'random_people'
train_endpoint = '/face/v1.0/persongroups/{0}/train'.format(person_group_id)

try:
    conn = http.client.HTTPSConnection(url)
    # bodyは空白
    conn.request('POST', train_endpoint, '', train_headers)
    response = conn.getresponse()
    data = response.read()
    print(data)
    conn.close()
except Exception as ex:
    print(ex)

b''


In [14]:
# 最初に使ったコードでface_idが検出できる
import urllib.parse, http.client, json

headers = {
    'Ocp-Apim-subscription-Key':subscription_key,
    'Content-Type':'application/json'
}

body = {
    'url':'https://upload.wikimedia.org/wikipedia/commons/thumb/0/0f/Angela_Merkel_2019_cropped.jpg/1200px-Angela_Merkel_2019_cropped.jpg'
}

# returnFaceIdをTrueに変更
params = urllib.parse.urlencode({
    'returnFaceId':True,
    'returnFaceLandmarks':True,
    'returnFaceAttributes':'age,gender,makeup,smile,glasses,accessories,emotion'
})

url = 'southcentralus.api.cognitive.microsoft.com'

endpoint = '/face/v1.0/detect?%s'

try:
    conn = http.client.HTTPSConnection(url)
    conn.request('POST', endpoint % params, str(body), headers)
    response = conn.getresponse()
    jsonData = response.read()
    data = json.loads(jsonData)
    detect_face_id = data[0]['faceId']
#     print(json.dumps(data, sort_keys=True, indent=2))
    conn.close()
except Exception as ex:
    print(ex)
print(detect_face_id)

9beb7230-ae2a-4837-bff1-421a303fbf73


In [15]:
# face_idから候補を検索
person_group_id = 'random_people'

identify_endpoint = '/face/v1.0/identify'

identify_headers = {
    'Ocp-Apim-subscription-Key':subscription_key,
}

identify_body = {
    'faceIds':[detect_face_id],
    'personGroupId':person_group_id
}

try:
    conn = http.client.HTTPSConnection(url)
    conn.request('POST', identify_endpoint, str(identify_body), identify_headers)
    response = conn.getresponse()
    jsonData = response.read()
    identify_data = json.loads(jsonData)
    print(identify_data)
    conn.close()
except Exception as ex:
    print(ex)

[{'faceId': '9beb7230-ae2a-4837-bff1-421a303fbf73', 'candidates': [{'personId': '79f0b49d-3ba3-40ac-863a-115f7056b907', 'confidence': 0.8039}]}]


In [16]:
# person_dataと比較
found = False
identified_person = identify_data[0]
for candidate in identified_person['candidates']:
    # person_dataはperson_IDとface_IDが入っている
    for person in person_data:
        if(person['personId'] == candidate['personId']):
            print('FOUND IT')
            print('This is ' + person['name'])
            found = True

if(found == False):
    print("We couldn't find this person")

FOUND IT
This is Angela
